<center><h1>Saloni_Fnu_HW3</h1></center>
<br>
<br>

Name: Fnu Saloni
<br>
Github Username: fnusaloni
<br>
USC ID: 3175644840

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [6]:
import os
import pandas as pd
import numpy as np

Get the AReM Data Set

In [8]:
location = '../data/AReM'

def getData(location):
    # combining all the dataframes from different files into one
    return pd.concat([
        # opening and reading each file
        pd.read_csv(os.path.join(location, activityName, file), 
                    # skiiping forst 5 rows as it doesnot contain any data and giving names to each column properly
                    skiprows=5, header=None, 
                    names=['time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23'], 
                    # since dataset4 of bending has the values in one colu only so using if for that else for rest dataset
                    sep=r"\s+" if (activityName == 'bending2' and file == 'dataset4.csv') else ",",
                    on_bad_lines='skip')
        .assign(activityName=activityName, dataset=file)  # adding activity name and dataset name in the dataframe extra
        # going through activity folders first
        for activityName in os.listdir(location) if os.path.isdir(os.path.join(location, activityName))
        # then going through csv files in each activity folder
        for file in os.listdir(os.path.join(location, activityName)) if file.startswith('dataset') and file.endswith('.csv')
    ], ignore_index=True)

dataRead = getData(location)
dataRead

,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,activityName,dataset
0,0,42.00,0.00,18.50,0.50,12.00,0.00,bending1,dataset7.csv
1,250,42.00,0.00,18.00,0.00,11.33,0.94,bending1,dataset7.csv
2,500,42.75,0.43,16.75,1.79,18.25,0.43,bending1,dataset7.csv
3,750,42.50,0.50,16.75,0.83,19.00,1.22,bending1,dataset7.csv
4,1000,43.00,0.82,16.25,0.83,18.00,0.00,bending1,dataset7.csv
...,...,...,...,...,...,...,...,...,...
42232,118500,40.25,1.30,12.67,1.70,20.50,0.50,cycling,dataset9.csv
42233,118750,41.75,1.79,11.00,6.00,16.67,2.49,cycling,dataset9.csv
42234,119000,36.33,0.47,16.00,3.16,20.33,1.70,cycling,dataset9.csv
42235,119250,31.50,1.50,21.00,0.00,12.25,7.12,cycling,dataset9.csv


In [9]:
# seeing the data for bending 2's dataset 4 that is it proper or not
bending2Ds4 = dataRead[(dataRead['activityName'] == 'bending2') & (dataRead['dataset'] == 'dataset4.csv')]

print("\nFiltered Dataset (Only bending2, dataset4.csv):")
print(bending2Ds4)


Filtered Dataset (Only bending2, dataset4.csv):
         time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
11040       0      32.50       0.50       0.00       0.00      19.00   
11041     250      32.50       0.50       0.00       0.00      18.50   
11042     500      32.75       0.43       1.00       0.00      18.00   
11043     750      32.50       0.50       0.00       0.00      17.50   
11044    1000      32.50       0.50       7.50       0.50      17.50   
...       ...        ...        ...        ...        ...        ...   
11515  118750      28.67       0.47       4.67       1.25      17.33   
11516  119000      27.50       0.50       5.50       2.50      17.25   
11517  119250      28.00       0.00       6.67       0.94      17.00   
11518  119500      28.00       0.00       5.00       0.82      17.00   
11519  119750      28.00       0.00       0.00       0.00      17.00   

       var_rss23 activityName       dataset  
11040       1.00     bending2  dataset4.

### (b) Test and Train Data

In [11]:
location = '../data/AReM'

# making the activity list
activitiesGiven = ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']

# keeping datasets 1 and 2 in folders bending1 and bending2, as well as datasets 1, 2, and 3 in other folders as test data
testData = {}
for activity in activitiesGiven:
    if activity in ['bending1', 'bending2']:
        testData[activity] = [1, 2]  
    else:
        testData[activity] = [1, 2, 3]

# considering other datasets as train data
trainData = {
    'bending1': list(range(3, 8)),
    'bending2': list(range(3, 7)),
}
for activity in activitiesGiven:
    if activity not in trainData:
        trainData[activity] = list(range(4, 16))  

# using this function to gather training or testing data in one place
def gatherDataset(dataType, isTrainData=False):
    dataframes = []
    fileNames = []
    
    # counting the number of dataset files for training or testing data
    datasetTotalCount = sum(len(datasetIds) for datasetIds in dataType.values())

    for activityName, datasetIds in dataType.items():
        for datasetId in datasetIds:
            # getting the path to each file
            fileLoc = os.path.join(location, activityName, f'dataset{datasetId}.csv')
            
            # since dataset4 of bending2 has values in one column only, handling it separately
            if isTrainData and activityName == 'bending2' and datasetId == 4:
                df = pd.read_csv(fileLoc, skiprows=5, header=None, 
                                 names=['time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23'], 
                                 sep=r"\s+", engine='python', on_bad_lines='skip')
            else:
                df = pd.read_csv(fileLoc, skiprows=5, header=None, 
                                 names=['time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23'], 
                                 sep=",", on_bad_lines='skip')

            # adding activity name as an extra column
            df['activityName'] = activityName  
            dataframes.append(df)
            fileNames.append(fileLoc)

    # merging all datasets into one
    return pd.concat(dataframes, ignore_index=True), fileNames, datasetTotalCount

# calling the function twice to load training and testing data
trainDataframe, trainFileName, trainIns = gatherDataset(trainData, isTrainData=True)
testDataframe, testFileName, testIns = gatherDataset(testData, isTrainData=False)

# showing the result
print(f"Training instances: {trainIns} \nTest instances: {testIns}")
print(f"\nTotal instances: {trainIns + testIns}")
print(f"\nTraining files: {trainFileName}")
print(f"\nTesting files: {testFileName}")

print("\nTesting Data:")
print(testDataframe)


Training instances: 69 
Test instances: 19

Total instances: 88

Training files: ['../data/AReM/bending1/dataset3.csv', '../data/AReM/bending1/dataset4.csv', '../data/AReM/bending1/dataset5.csv', '../data/AReM/bending1/dataset6.csv', '../data/AReM/bending1/dataset7.csv', '../data/AReM/bending2/dataset3.csv', '../data/AReM/bending2/dataset4.csv', '../data/AReM/bending2/dataset5.csv', '../data/AReM/bending2/dataset6.csv', '../data/AReM/cycling/dataset4.csv', '../data/AReM/cycling/dataset5.csv', '../data/AReM/cycling/dataset6.csv', '../data/AReM/cycling/dataset7.csv', '../data/AReM/cycling/dataset8.csv', '../data/AReM/cycling/dataset9.csv', '../data/AReM/cycling/dataset10.csv', '../data/AReM/cycling/dataset11.csv', '../data/AReM/cycling/dataset12.csv', '../data/AReM/cycling/dataset13.csv', '../data/AReM/cycling/dataset14.csv', '../data/AReM/cycling/dataset15.csv', '../data/AReM/lying/dataset4.csv', '../data/AReM/lying/dataset5.csv', '../data/AReM/lying/dataset6.csv', '../data/AReM/lying/d

In [12]:
# seeing train data
print("Train Data:")
trainDataframe

Train Data:


,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,activityName
0,0,42.00,0.71,21.25,0.43,30.00,0.00,bending1
1,250,41.50,0.50,20.25,1.48,31.25,1.09,bending1
2,500,41.50,0.50,14.25,1.92,33.00,0.00,bending1
3,750,40.75,0.83,15.75,0.43,33.00,0.00,bending1
4,1000,40.00,0.71,20.00,2.74,32.75,0.43,bending1
...,...,...,...,...,...,...,...,...
33112,118750,34.50,6.18,9.00,3.56,12.67,4.19,walking
33113,119000,25.75,6.02,13.75,2.05,16.00,1.58,walking
33114,119250,31.50,3.35,10.25,5.12,16.25,2.95,walking
33115,119500,33.75,2.77,14.00,3.24,13.75,0.43,walking


### (c) Feature Extraction

#### i. Research

The types of time-domain features are usually used in time series classification are:
a) Mean: It is the average value of the dataset.                                         
b) Median: It is the middle value when the data is arranged in order.                    
c) Mode: It is the most frequently occurring value in the dataset.                       
d) Maximum and Minimum: It is the highest and lowest values in the dataset helping to identify extremes.                                                                      
e) Interquartile Range (IQR): It focuses on the middle 50% of the data ignoring extreme values that might distort the analysis.                                                
f) Range: It is the difference between the max and min values showing how wide the spread is.                                                                           
g) Variance: It measures how spread out the data is from the mean. High variance means values fluctuate a lot, while low variance means they stay close to the mean.         
h) Standard Deviation: It is an another way to measure spread and is just the square root of variance.                                                                     
i) Percentiles: It splits the data into sections just like the median in order to understand how values are distributed.                                              
j) Autocorrelation: It checks how similar the data is to its past values and is great for finding repeating patterns, like daily or seasonal trends.                       
k) Zero-Crossing Rate: It counts how often the signal changes from positive to negative or vice versa.                                                                     
l) Root Mean Square (RMS): It measures the overall energy or intensity of the data.      
m) Kurtosis: It helps us understand if the data has a sharp peak ie many values near the center or if it’s more spread out.                                                

#### ii. Extraction

In [17]:
# List of column names
columnName = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']

# calculating statistical features for the dataset
def calculation(dataRead):
    # grouping the data by dataset and activityName
    grpData = dataRead.groupby(['dataset', 'activityName'])[columnName]

    # calculating statistical features for each group
    newDataset = grpData.agg([
        ('min', 'min'),
        ('max', 'max'),
        ('mean', 'mean'),
        ('median', 'median'),
        ('std', 'std'),
        ('1st_quart', lambda x: x.quantile(0.25)),
        ('3rd_quart', lambda x: x.quantile(0.75))
    ]).reset_index()

    # formatting column names
    newDataset.columns = ['_'.join(col).strip('_') for col in newDataset.columns]

    # renaming columns for proper presentation
    renameCol = {}
    for x, colName in enumerate(columnName):
        for stat in ['min', 'max', 'mean', 'median', 'std', '1st_quart', '3rd_quart']:
            renameCol[f"{colName}_{stat}"] = f"{stat}_{x+1}"

    # renaming dataset
    newDataset.rename(columns=renameCol, inplace=True)

    return newDataset

# using the calculation function for dataRead
calculatedDataset = calculation(dataRead)
calculatedDataset


,dataset,activityName,min_1,max_1,mean_1,median_1,std_1,1st_quart_1,3rd_quart_1,min_2,...,std_5,1st_quart_5,3rd_quart_5,min_6,max_6,mean_6,median_6,std_6,1st_quart_6,3rd_quart_6
0,dataset1.csv,bending1,37.25,45.00,40.624792,40.50,1.476967,39.2500,42.00,0.0,...,2.188449,33.0000,36.00,0.0,1.92,0.570583,0.43,0.582915,0.00,1.3000
1,dataset1.csv,bending2,12.75,51.00,24.562958,24.25,3.737514,23.1875,26.50,0.0,...,3.693786,20.5000,27.00,0.0,4.97,0.700187,0.50,0.693720,0.43,0.8700
2,dataset1.csv,cycling,24.25,45.00,37.177042,36.25,3.581301,34.5000,40.25,0.0,...,2.890347,17.9500,21.75,0.0,9.34,2.921729,2.50,1.852600,1.50,3.9000
3,dataset1.csv,lying,23.50,30.00,27.716375,27.50,1.442253,27.0000,29.00,0.0,...,4.074511,5.5000,10.75,0.0,4.50,0.734271,0.71,0.613688,0.43,1.0000
4,dataset1.csv,sitting,33.25,48.00,42.363562,43.00,2.068247,42.0000,43.50,0.0,...,4.956821,10.1875,17.69,0.0,6.02,1.034021,0.83,0.985627,0.47,1.2500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,dataset9.csv,cycling,27.75,44.67,37.142359,36.33,3.762442,34.0000,40.50,0.0,...,2.687173,15.0000,18.75,0.0,8.75,2.825720,2.59,1.637312,1.59,3.7400
84,dataset9.csv,lying,39.00,56.25,47.325125,42.67,5.961280,42.0000,54.00,0.0,...,4.142132,11.7500,18.00,0.0,5.72,0.766167,0.50,0.723953,0.43,1.0000
85,dataset9.csv,sitting,41.75,46.50,43.190854,42.50,1.230345,42.0000,44.50,0.0,...,3.027169,14.0000,18.33,0.0,5.45,0.806229,0.82,0.645476,0.43,1.0900
86,dataset9.csv,standing,36.33,47.67,45.399625,45.50,1.328121,45.0000,46.33,0.0,...,3.374095,11.2500,14.50,0.0,4.50,0.795104,0.82,0.503007,0.47,1.0000


#### iii. Standard Deviation

In [19]:
# selecting only the numeric columns
numFeature = calculatedDataset.select_dtypes(include=[np.number])

# # calculating standard deviation for each feature
# def calStd(dataRead):
#     return dataRead.std()

# calculating bootstrap CI
def calBootCI(dataRead, numBootstrap=1000, confidenceLevel=0.90):
    bootStds = np.zeros(numBootstrap)  

    # randomly selecting data points from dataRead with replacement to create a new sample and calculating standard deviation for that
    for x in range(numBootstrap):
        select = np.random.choice(dataRead, size=len(dataRead), replace=True)  
        bootStds[x] = np.std(select)  

    # calculating CI
    lowerCi = np.percentile(bootStds, 5)  # 5th percentile
    upperCi = np.percentile(bootStds, 95)  # 95th percentile

    return lowerCi, upperCi

# calculating standard deviation for each feature
origStds = numFeature.std()

# calculating bootstrap CI for each feature
CI = np.array([calBootCI(numFeature[col]) for col in numFeature.columns])

# converting CI into a dataframe
CIDataframe = pd.DataFrame(CI, index=numFeature.columns, columns=["Lower CI", "Upper CI"])


print("\nOriginal Standard Deviations:")
print(origStds)

print("\nBootstrap 90% Confidence Intervals for Standard Deviations:")
print(CIDataframe)



Original Standard Deviations:
min_1          9.569975
max_1          4.394362
mean_1         5.335700
median_1       5.440054
std_1          1.772185
1st_quart_1    6.153874
3rd_quart_1    5.138925
min_2          0.000000
max_2          5.062729
mean_2         1.574198
median_2       1.412293
std_2          0.884137
1st_quart_2    0.946386
3rd_quart_2    2.125399
min_3          2.956462
max_3          4.875137
mean_3         4.008228
median_3       4.036396
std_3          0.946670
1st_quart_3    4.220658
3rd_quart_3    4.171628
min_4          0.000000
max_4          2.183625
mean_4         1.166178
median_4       1.145985
std_4          0.458283
1st_quart_4    0.843405
3rd_quart_4    1.552504
min_5          6.124001
max_5          5.741238
mean_5         5.675543
median_5       5.813782
std_5          1.024918
1st_quart_5    6.096465
3rd_quart_5    5.531720
min_6          0.045838
max_6          2.518921
mean_6         1.154889
median_6       1.086474
std_6          0.517651
1st_quart

#### iv. Select Features

In [21]:
# removing non numeric columns
numberFeature = calculatedDataset.select_dtypes(include=[float, int])

# grouping by activity label and calculating three key statistics for each feature:
statistics = numberFeature.groupby(calculatedDataset["activityName"]).agg(["mean", "std", lambda x: x.max() - x.min()])

# renaming the columns
statistics.columns = ["_".join(col).strip() for col in statistics.columns]

# showing the statistics
print("\nFeature Statistics - Mean, Std, Range per Activity :")
statistics



Feature Statistics - Mean, Std, Range per Activity :


,min_1_mean,min_1_std,min_1_<lambda_0>,max_1_mean,max_1_std,max_1_<lambda_0>,mean_1_mean,mean_1_std,mean_1_<lambda_0>,median_1_mean,...,median_6_<lambda_0>,std_6_mean,std_6_std,std_6_<lambda_0>,1st_quart_6_mean,1st_quart_6_std,1st_quart_6_<lambda_0>,3rd_quart_6_mean,3rd_quart_6_std,3rd_quart_6_<lambda_0>
activityName,,,,,,,,,,,,,,,,,,,,,
bending1,35.642857,2.030277,5.00,46.795714,1.275524,3.00,42.667723,1.250419,3.344333,42.904286,...,0.070,0.531610,0.079900,0.233369,0.061429,0.162525,0.430,0.962857,0.279804,0.8000
bending2,17.541667,9.995520,27.50,44.958333,6.581065,18.00,29.415344,2.920310,8.023250,28.993333,...,0.440,0.905411,0.301775,0.851877,0.468333,0.022286,0.070,1.168333,0.205175,0.4300
cycling,24.377333,3.358966,10.25,44.483333,0.476140,1.50,36.747290,0.520192,1.808833,36.221667,...,0.890,1.749547,0.111433,0.395348,1.721167,0.230244,0.760,4.038333,0.281337,0.9925
lying,33.216667,12.334934,48.00,42.733333,8.639697,26.25,39.850217,7.142058,20.287792,39.278000,...,0.240,0.560337,0.119245,0.335581,0.340833,0.179044,0.460,0.896667,0.080858,0.1700
sitting,35.544667,6.528011,19.00,46.356000,2.908998,12.08,41.748634,3.424276,13.011688,41.359000,...,0.590,0.821134,0.150418,0.542337,0.445333,0.023563,0.070,1.145000,0.194008,0.8700
standing,34.744000,2.284650,7.25,47.120000,0.916110,2.75,43.857747,1.415475,4.512292,44.412000,...,0.590,0.710320,0.143333,0.455506,0.463333,0.023197,0.070,1.165667,0.199453,0.7550
walking,18.816000,2.789475,11.00,46.261333,2.274938,7.75,34.433676,0.457919,1.683604,35.069333,...,0.305,1.699922,0.057348,0.174800,2.105833,0.082048,0.355,4.408167,0.108642,0.3250


Based on the values observed from the Feature Summary Statistics the three most important time-domain features are:

a) Mean: The mean value represents the overall average signal strength for each activity. From the table we can see that activities like bending1 (42.67) and walking (34.43) have different mean values, which helps differentiate between high and low-intensity activities.

b) Standard Deviation: The standard deviation tells us how much the signal fluctuates. From the table we can see tha activities like bending2 (9.99) and lying (12.33) have high standard deviation, meaning these activities have high variations in movement while cycling (0.52) and walking (0.45) have low standard deviation, meaning the signal is more stable. This feature is useful because it helps differentiate dynamic and static activities.    

c) Median: The median represents the middle value of the dataset, giving insight into the data's distribution.From the table we can see tha activities bending2 (28.99) has a lower median compared to its mean (29.41), which suggests the presence of some high-value spikes in the data.If the mean and median are close, the data is evenly spread. If they are far apart, the data is skewed.

## 2. ISLR 3.7.4

### (a) Linear Train

The cubic model is more flexible than the linear model since it contains more terms, thus it will fit training data as good as, or even better than, the linear model. If the true relationship is linear, the cubic model can still provide slightly lower RSS by fitting noise. If it is nonlinear, the cubic model will provide lower RSS by fitting curves. But such flexibility can lead to overfitting, i.e., the training RSS for the cubic model is always equal or less but not necessarily superior.

### (b) Linear Test

The linear model should have a lower test RSS since the true relationship is linear. The cubic model, with more parameters, may overfit the training data, capturing noise as opposed to the true pattern. Thus, it may fail to generalize well to new data, resulting in a higher test RSS compared to the linear model.

### (c) Not Linear Train

The cubic model will have a lower or the same training RSS since its extra parameters enable it to fit data more accurately. The linear one may not be able to capture all the patterns if the real relationship is not precisely linear, thus it will have a higher RSS. The cubic model, while capable of fitting non-linear trends, can also overfit, so the lower RSS is less important.

### (d) Not Linear Testing

Since there is no clear information on the true relationship between X and Y, I can’t determine which model will perform better. The model that best matches the actual relationship will have a lower test RSS. If the relationship is closer to linear, the linear model will perform better. If it is closer to cubic, the cubic model will have a lower RSS.

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 

## Reference

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html                              https://www.geeksforgeeks.org/python-os-path-join-method/                                           https://www.geeksforgeeks.org/python-os-listdir-method                                              https://stackoverflow.com/questions/62867315/python-script-for-startswith-and-endswith              https://stackoverflow.com/questions/57023018/python-list-comprehension-execution-order/57023028
https://docs.python.org/3/library/functions.html#enumerate
https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html
https://docs.python.org/2/library/stdtypes.html#str.join
https://www.geeksforgeeks.org/numpy-zeros-python/
https://numpy.org/doc/2.1/reference/generated/numpy.percentile.html
https://stackoverflow.com/questions/13959510/python-list-initialization-using-multiple-range-statements
